In [ ]:
from dash import Dash, dcc, html, Input, Output, State, dash_table
import plotly.graph_objs as go
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import base64
import io

class HealthMetricsDashboard:
    def __init__(self):
        # Initialize Dash app
        self.app = Dash(__name__)
        self.df = self.load_default_data()
        self.prepare_data()
        self.layout()
        self.register_callbacks()

    def load_default_data(self):
        # Load the default CSV file
        default_csv = 'current.csv'  # Replace with your default CSV file path
        df = pd.read_csv(default_csv)
        df['ActivityDate'] = pd.to_datetime(df['ActivityDate'])
        return df

    def parse_contents(self, contents, filename):
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        try:
            if 'csv' in filename:
                # Assume that the user uploaded a CSV file
                df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
                df['ActivityDate'] = pd.to_datetime(df['ActivityDate'])
                return df
            else:
                return None
        except Exception as e:
            print(e)
            return None

    def prepare_data(self):
        # Prepare data
        self.df['TotalActiveMinutes'] = self.df['VeryActiveMinutes'] + \
            self.df['FairlyActiveMinutes'] + self.df['LightlyActiveMinutes']
        self.df['TotalHours'] = self.df['TotalActiveMinutes'] / 60  # Calculating active hours
        self.df['WeekDays'] = self.df['ActivityDate'].dt.day_name()

        # Calculate weekly averages
        self.weekly_averages = self.df.groupby('WeekDays').agg({
            'TotalSteps': 'mean',
            'TotalActiveMinutes': 'mean',
            'Calories': 'mean'
        }).reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

    def layout(self):
        # Layout of the Dash app
        self.app.layout = html.Div([
            html.H1("Health Metrics Dashboard"),

            # File upload component
            html.H2("Upload CSV File"),
            dcc.Upload(
                id='upload-data',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select a CSV File')
                ]),
                style={
                    'width': '100%',
                    'height': '60px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px'
                },
                multiple=False
            ),
            html.Div(id='output-data-upload'),

            # Dropdown and graph for bar chart
            html.Label("Select Weekday for Bar Chart"),
            dcc.Dropdown(
                id='weekday-dropdown',
                options=[{'label': day, 'value': day} for day in self.df['WeekDays'].unique()],
                value=['Monday'],  # Default selection
                multi=True
            ),
            dcc.Graph(id="bar-chart"),

            # Dropdown and sliders for weekly averages graph
            html.Label("Select Weekday for Weekly Averages"),
            dcc.Dropdown(
                id='weekly-avg-weekday-dropdown',
                options=[{'label': day, 'value': day} for day in self.weekly_averages.index],
                value=self.weekly_averages.index.tolist(),  # Default selection for all days
                multi=True
            ),
            html.Label("Filter by Calories"),
            dcc.RangeSlider(
                id="calories-range-slider",
                min=0,
                max=self.df['Calories'].max(),
                step=10,
                value=[0, self.df['Calories'].max()],
                marks={int(cal): str(cal) for cal in range(0, int(self.df['Calories'].max()) + 1, 100)}
            ),
            html.Label("Filter by Steps"),
            dcc.RangeSlider(
                id="steps-range-slider",
                min=0,
                max=self.df['TotalSteps'].max(),
                step=1000,
                value=[0, self.df['TotalSteps'].max()],
                marks={int(step): str(step) for step in range(0, int(self.df['TotalSteps'].max()) + 1, 5000)}
            ),
            dcc.Graph(id="weekly-averages-graph"),

            # Heatmap
            html.Label("Select Metric for Heatmap"),
            dcc.Dropdown(
                id='heatmap-dropdown',
                options=[{'label': col, 'value': col} for col in [
                    'TotalSteps', 'TotalActiveMinutes', 'Calories', 'VeryActiveMinutes',
                    'FairlyActiveMinutes', 'LightlyActiveMinutes', 'SedentaryMinutes']],
                value=['TotalSteps', 'TotalActiveMinutes', 'Calories'],  # Initial metrics
                multi=True
            ),
            html.Label("Adjust Color Bar Range"),
            dcc.Slider(id="heatmap-slider", min=0, max=1, step=0.1, value=0.5),
            dcc.Graph(id="heatmap"),

            # Scatter plots
            html.Label("Filter Scatter Plot by Calories Range"),
            dcc.RangeSlider(
                id="calories-slider",
                min=self.df['Calories'].min(),
                max=self.df['Calories'].max(),
                step=10,
                value=[self.df['Calories'].min(), self.df['Calories'].max()],  # Initial range
                marks={int(cal): str(cal) for cal in range(
                    int(self.df['Calories'].min()), int(self.df['Calories'].max()) + 1, 100)}
            ),
            dcc.Graph(id="scatter-plot"),

            html.Label("Filter Sedentary vs Active Scatter Plot by Total Active Minutes Range"),
            dcc.RangeSlider(
                id="active-minutes-slider",
                min=self.df['TotalActiveMinutes'].min(),
                max=self.df['TotalActiveMinutes'].max(),
                step=10,
                value=[self.df['TotalActiveMinutes'].min(), self.df['TotalActiveMinutes'].max()],
                marks={int(active): str(active) for active in range(
                    int(self.df['TotalActiveMinutes'].min()), int(self.df['TotalActiveMinutes'].max()) + 1, 100)}
            ),
            dcc.Graph(id="sedentary-vs-active-scatter"),

            # Pie chart
            html.Label("Select View for Pie Chart"),
            dcc.RadioItems(
                id='pie-radio',
                options=[
                    {'label': 'Total View', 'value': 'total'},
                    {'label': 'Active Minutes View', 'value': 'active'}
                ],
                value='total',
                labelStyle={'display': 'inline-block', 'margin-right': '10px'}
            ),
            dcc.Graph(id="pie-chart"),

            html.Label("Select Date Range for Time Series Graph"),
            dcc.DatePickerRange(
                id="date-range-picker",
                start_date=self.df['ActivityDate'].min(),
                end_date=self.df['ActivityDate'].max(),
                display_format="YYYY-MM-DD",
            ),
            html.H2("Health Metrics Over Time"),
            dcc.Graph(id="health-metrics-time-series"),
        ])

    def register_callbacks(self):
        # Callback for file upload
        @self.app.callback(
            Output('output-data-upload', 'children'),
            Output('weekday-dropdown', 'options'),
            Output('weekly-avg-weekday-dropdown', 'options'),
            Output('calories-range-slider', 'max'),
            Output('calories-range-slider', 'marks'),
            Output('calories-slider', 'min'),
            Output('calories-slider', 'max'),
            Output('calories-slider', 'marks'),
            Output('steps-range-slider', 'max'),
            Output('steps-range-slider', 'marks'),
            Output('active-minutes-slider', 'min'),
            Output('active-minutes-slider', 'max'),
            Output('active-minutes-slider', 'marks'),
            Output('date-range-picker', 'start_date'),
            Output('date-range-picker', 'end_date'),
            Input('upload-data', 'contents'),
            State('upload-data', 'filename')
        )
        def update_output(contents, filename):
            if contents is not None:
                new_df = self.parse_contents(contents, filename)
                if new_df is not None:
                    self.df = new_df
                    self.prepare_data()
                    children = html.Div([
                        html.H5(filename),
                        dash_table.DataTable(
                            data=self.df.head(5).to_dict('records'),
                            columns=[{'name': i, 'id': i} for i in self.df.columns]
                        )
                    ])
                    # Update dropdown options and slider ranges based on new data
                    weekday_options = [{'label': day, 'value': day} for day in self.df['WeekDays'].unique()]
                    weekly_avg_weekday_options = [{'label': day, 'value': day} for day in self.weekly_averages.index]
                    calories_max = self.df['Calories'].max()
                    calories_marks = {int(cal): str(cal) for cal in range(0, int(calories_max) + 1, 100)}
                    calories_slider_min = self.df['Calories'].min()
                    calories_slider_max = self.df['Calories'].max()
                    calories_slider_marks = {int(cal): str(cal) for cal in range(
                        int(calories_slider_min), int(calories_slider_max) + 1, 100)}
                    steps_max = self.df['TotalSteps'].max()
                    steps_marks = {int(step): str(step) for step in range(0, int(steps_max) + 1, 5000)}
                    active_min = self.df['TotalActiveMinutes'].min()
                    active_max = self.df['TotalActiveMinutes'].max()
                    active_marks = {int(active): str(active) for active in range(
                        int(active_min), int(active_max) + 1, 100)}
                    start_date = self.df['ActivityDate'].min()
                    end_date = self.df['ActivityDate'].max()
                    return (children, weekday_options, weekly_avg_weekday_options,
                            calories_max, calories_marks, calories_slider_min, calories_slider_max,
                            calories_slider_marks, steps_max, steps_marks,
                            active_min, active_max, active_marks, start_date, end_date)
            # If no file is uploaded, return defaults
            return (html.Div(), dash.no_update, dash.no_update, dash.no_update, dash.no_update,
                    dash.no_update, dash.no_update, dash.no_update, dash.no_update, dash.no_update,
                    dash.no_update, dash.no_update, dash.no_update, dash.no_update, dash.no_update)

        # Callback for bar chart
        @self.app.callback(
            Output("bar-chart", "figure"),
            Input("weekday-dropdown", "value")
        )
        def update_bar_chart(selected_days):
            f1_group = self.df[self.df['WeekDays'].isin(selected_days)].groupby('WeekDays').agg({
                'TotalSteps': 'sum', 'Calories': 'sum'}).reset_index()
            fig = px.bar(f1_group, x='WeekDays', y='Calories', color='WeekDays',
                         text='TotalSteps', title='Calories Burned by Total Steps Across Weekdays')
            return fig

        # Callback for heatmap
        @self.app.callback(
            Output("heatmap", "figure"),
            [Input("heatmap-dropdown", "value"), Input("heatmap-slider", "value")]
        )
        def update_heatmap(selected_metrics, cbar_range):
            correlation_matrix = self.df[selected_metrics].corr()
            fig = go.Figure(data=go.Heatmap(
                z=correlation_matrix.values,
                x=correlation_matrix.columns,
                y=correlation_matrix.index,
                colorscale='RdBu',
                zmin=-cbar_range, zmax=cbar_range
            ))
            fig.update_layout(title="Correlation Matrix of Health Metrics")
            return fig

        # Callback for scatter plots
        @self.app.callback(
            [Output("scatter-plot", "figure"), Output("sedentary-vs-active-scatter", "figure")],
            [Input("calories-slider", "value"), Input("active-minutes-slider", "value")]
        )
        def update_scatter_plots(calories_range, active_minutes_range):
            min_calories, max_calories = calories_range
            min_active, max_active = active_minutes_range

            filtered_df_calories = self.df[(self.df['Calories'] >= min_calories) & (self.df['Calories'] <= max_calories)]
            filtered_df_active = self.df[(self.df['TotalActiveMinutes'] >= min_active) & (self.df['TotalActiveMinutes'] <= max_active)]

            scatter_fig = make_subplots(rows=1, cols=3, subplot_titles=(
                "Total Steps vs Calories",
                "Total Active Minutes vs Calories",
                "Very Active Minutes vs Calories"
            ))
            scatter_fig.add_trace(go.Scatter(
                x=filtered_df_calories['TotalSteps'], y=filtered_df_calories['Calories'],
                mode='markers', name="Total Steps vs Calories"
            ), row=1, col=1)
            scatter_fig.add_trace(go.Scatter(
                x=filtered_df_calories['TotalActiveMinutes'], y=filtered_df_calories['Calories'],
                mode='markers', name="Total Active Minutes vs Calories"
            ), row=1, col=2)
            scatter_fig.add_trace(go.Scatter(
                x=filtered_df_calories['VeryActiveMinutes'], y=filtered_df_calories['Calories'],
                mode='markers', name="Very Active Minutes vs Calories"
            ), row=1, col=3)
            scatter_fig.update_layout(title="Scatter Plots of Activity Metrics and Calories")

            sedentary_vs_active_fig = go.Figure()
            sedentary_vs_active_fig.add_trace(go.Scatter(
                x=filtered_df_active['TotalActiveMinutes'],
                y=filtered_df_active['SedentaryMinutes'],
                mode='markers',
                marker=dict(color='magenta', opacity=0.6),
                name="Sedentary vs Active"
            ))
            sedentary_vs_active_fig.update_layout(
                title='Comparison of Sedentary Minutes vs. Total Active Minutes',
                xaxis_title='Total Active Minutes',
                yaxis_title='Sedentary Minutes',
                title_x=0.5
            )

            return scatter_fig, sedentary_vs_active_fig

        # Callback for pie chart
        @self.app.callback(
            Output("pie-chart", "figure"),
            Input("pie-radio", "value")
        )
        def update_pie_chart(_):
            # Create the data structure for the pie chart
            pie_data = pd.DataFrame({
                'Activity Type': ['VeryActiveMinutes', 'FairlyActiveMinutes', 'LightlyActiveMinutes', 'SedentaryMinutes'],
                'Time': [
                    self.df['VeryActiveMinutes'].sum(),
                    self.df['FairlyActiveMinutes'].sum(),
                    self.df['LightlyActiveMinutes'].sum(),
                    self.df['SedentaryMinutes'].sum()
                ]
            })

            # Generate pie chart
            fig = px.pie(pie_data, names='Activity Type', values='Time', title='Activity Time Distribution')
            fig.update_layout(height=600, width=800, title_x=0.5)
            return fig

        # Callback for weekly averages graph
        @self.app.callback(
            Output("weekly-averages-graph", "figure"),
            [Input("weekly-avg-weekday-dropdown", "value"),
             Input("calories-range-slider", "value"),
             Input("steps-range-slider", "value")]
        )
        def update_weekly_averages(selected_days, calories_range, steps_range):
            min_calories, max_calories = calories_range
            min_steps, max_steps = steps_range

            filtered_df = self.df[(self.df['WeekDays'].isin(selected_days)) &
                                  (self.df['Calories'] >= min_calories) &
                                  (self.df['TotalSteps'] >= min_steps) &
                                  (self.df['TotalSteps'] <= max_steps)]

            filtered_weekly_averages = filtered_df.groupby('WeekDays').agg({
                'TotalSteps': 'mean',
                'TotalActiveMinutes': 'mean',
                'Calories': 'mean'
            }).reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

            fig = go.Figure()
            fig.add_trace(go.Bar(
                x=filtered_weekly_averages.index,
                y=filtered_weekly_averages['TotalSteps'],
                name="Average Steps",
                marker_color='blue',
                opacity=0.6
            ))
            fig.add_trace(go.Scatter(
                x=filtered_weekly_averages.index,
                y=filtered_weekly_averages['TotalActiveMinutes'],
                name="Active Minutes",
                mode='lines+markers',
                marker=dict(color='green')
            ))
            fig.add_trace(go.Scatter(
                x=filtered_weekly_averages.index,
                y=filtered_weekly_averages['Calories'],
                name="Calories Burned",
                mode='lines+markers',
                marker=dict(color='red')
            ))
            fig.update_layout(
                title="Average Steps, Active Minutes, and Calories Burned by Weekday",
                xaxis_title="Weekday"
            )
            return fig

        # Callback for time series graph
        @self.app.callback(
            Output("health-metrics-time-series", "figure"),
            [Input("date-range-picker", "start_date"),
             Input("date-range-picker", "end_date")]
        )
        def update_time_series_graph(start_date, end_date):
            # Filter data based on selected date range
            filtered_df = self.df[(self.df['ActivityDate'] >= start_date) & (self.df['ActivityDate'] <= end_date)]

            fig = make_subplots(
                rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                subplot_titles=(
                    "Total Steps Over Time",
                    "Active Minutes Over Time",
                    "Active Hours Over Time",
                    "Calories Burned Over Time"
                )
            )

            fig.add_trace(go.Scatter(
                x=filtered_df['ActivityDate'], y=filtered_df['TotalSteps'],
                mode='lines+markers', name='Total Steps'), row=1, col=1)
            fig.add_trace(go.Scatter(
                x=filtered_df['ActivityDate'], y=filtered_df['TotalActiveMinutes'],
                mode='lines+markers', name='Active Minutes'), row=2, col=1)
            fig.add_trace(go.Scatter(
                x=filtered_df['ActivityDate'], y=filtered_df['TotalHours'],
                mode='lines+markers', name='Active Hours'), row=3, col=1)
            fig.add_trace(go.Scatter(
                x=filtered_df['ActivityDate'], y=filtered_df['Calories'],
                mode='lines+markers', name='Calories Burned'), row=4, col=1)

            # Updating axes
            fig.update_xaxes(title_text="Date", row=4, col=1)
            fig.update_yaxes(title_text="Steps", row=1, col=1)
            fig.update_yaxes(title_text="Minutes", row=2, col=1)
            fig.update_yaxes(title_text="Hours", row=3, col=1)
            fig.update_yaxes(title_text="Calories", row=4, col=1)

            # Layout settings
            fig.update_layout(
                height=900, width=800,
                title_text="Health Metrics Over Time",
                showlegend=True
            )
            return fig

    def run(self):
        self.app.run_server(debug=True, port=8052)

if __name__ == "__main__":
    dashboard = HealthMetricsDashboard()
    dashboard.run()
